# Data Scraping and Manipulation for Tour de France Capstone Project

## This workbook details:
### 1. Webscraping used to gather data from Pro Cycling Stats
### 2. Importing and combining Kaggle data to enrich the scraped data
### 3. Cleaning and manipulation the data ready for visualisation and modelling

In [1]:
## importing libraries

## dataframe & numerical minipulation 
import pandas as pd
import numpy as np
from math import sqrt
import datetime

## for webscraping & text cleaning
from bs4 import BeautifulSoup
import requests
import re

# Functions

In [2]:
def word_finder_team(text):
    '''
    Takes in text and returns all text ater ">
    '''
    a = re.search(r'">.*',text).group()
    return a
    
def clean_number(text):
    '''
    Takes in text and returns the digit characters.
    If there are no digit characters, returns N/A
    '''
    try:
        a = re.search(r'[0-9.,]+',text).group()
    except AttributeError: 
        a = 'N/A'
    return a

In [3]:
def standardise_name(name):
    '''
    For a text name, splits at a whitespace, reverses the order, and uppercases the new first word
    e.g. Mark Cavendish -> CAVENDISH Mark
    '''
    if ' ' in name:
        oldname = name.split(' ')
        surname = oldname[1].upper()
        firstname = oldname[0]
        fullname = surname + ' ' + firstname
    else:
        fullname = name
    return fullname

In [4]:
def format_list(list):
    '''
    Applies the standardise_name function to a list of names, returns a list of the modified names
    '''
    lst = []
    for y in list: 
        z = standardise_name(y)
        lst.append(z)
    return lst

In [5]:
def race_data(year,race):
    '''
    takes int int string, string, returns a list
    inputs:
    start year: from 1935 vuelta, 1903 tour, 1909 giro 
    end year: latest 2021
    race: vuelta-a-espana, tour-de-france, or giro-d-italia suggested
    output: lists of the rank, age, team, and time after finish between these years
    '''
    
    assert int(year) > 1935 and int(year) < datetime.datetime.now().year
    , 'Earliest year must be 1935, latest year must be current year'
    assert race in ('tour-de-france', 'vuelta-a-espana','giro-d-italia'), "races are 'tour-de-france','vuelta-a-espana','giro-d-italia'"
    
    link = 'https://www.procyclingstats.com/race/'+race+'/'+str(year)+'/gc/'
    result = requests.get(link)
    soup = BeautifulSoup(result.text, 'html.parser')

    ## Ranks
    rank = [x.td for x in soup.findAll('tr')]
    rank_nonone = list(filter(None, rank))
    ranks = []
    for n in rank_nonone:
        a = clean_number(str(n))
        ranks.append(a)
    index_from = [i for i, n in enumerate(ranks) if n == '1'][1]
    if [i for i, n in enumerate(ranks) if n == '1'][2] - index_from < 20:
        index_to = [i for i, n in enumerate(ranks) if n == '1'][3]
    else:
        index_to = [i for i, n in enumerate(ranks) if n == '1'][2]
    ranks = ranks[index_from : index_to]

    ## Names
    names = [x.a for x in soup.findAll('tr')]
    names_nonone = list(filter(None, names))
    rider = []
    for n in names_nonone:
        if ("rider" in str(n)) == True:
            a = word_finder_team(str(n))
            a = a.replace('">','')
            a = a.replace('</a>','')
            rider.append(a)
    rider = rider[index_from : index_to]

    ## Times
    time = [x.text for x in soup.findAll('td', class_='time ar')]
    time = time[index_from : index_to] 
    new_times = []
    for t in time: 
        a = t.replace(',,','error')
        a = t.replace(',','')
        a = a.replace(' ','')
        if len(a) > 8:
            middle = int(len(a)/2)
            a = a[:middle]
            new_times.append(a)
        else:
            new_times.append(a)

    finish_time = new_times.pop(0)
    new_times.insert(0,'0')
    time = []
    for t in new_times: 
        if len(t) > 7:
            middle = int(len(t)/2)
            t = t[:middle]
            time.append(t)
        else:
            time.append(t)

    ## Teams
    team = [x.text for x in soup.findAll('td', class_='cu600')]
    team = team[index_from : index_to]

    ## Ages
    age = [x.text for x in soup.findAll('td', class_="age hide")]
    age = age[index_from : index_to]
    
    ## Year column
    years = [year for x in range(len(ranks))]
    
    return years, ranks, rider, age, team, time

In [6]:
def race_data_multiple_years(start,end,race):
    '''
    Applies the race_data fuction over multiple years, returns a dataframe of all valid years,
    plus prints any years that errored such that this can be manually searched
    '''
    year = [] 
    rank = [] 
    rider = []
    age = []
    team = []
    time = []
    for n in range(start,end):
        y, ra, ri, a, te, ti = race_data(n,race)
        if len(y)==len(ra)==len(ri)==len(a)==len(te)==len(ti):
            year += y
            rank += ra
            rider += ri
            age += a
            team += te
            time += ti
        else:
            print(n)
    coldict = {'year': year, 'rank': rank, 'rider': rider, 'age':age, 'team':team, 'time':time} 
    tdf_rank = pd.DataFrame(coldict)
    return tdf_rank

In [7]:
def word_finder_team(text):
        a = re.search(r'">.*',text).group()
        return a
    
def top3teamsprev(year_start,year_end,race):
    '''
    takes int int string, returns list of lists
    inputs:
    enter start year: from 1935 vuelta, 1903 tour, 1909 giro 
    end year: latest 2021
    race: vuelta-a-espana, tour-de-france, or giro-d-italia suggested
    output: list of top 3 teams for a race between years provided
    '''
    teams = []
    years = [x for x in range(year_start,year_end+1)]
    for n in range(year_start,year_end+1):
        link = 'https://www.procyclingstats.com/race/'+race+'/'+str(n)
        result = requests.get(link)
        soup = BeautifulSoup(result.text, 'html.parser')
        souplist = [x.a for x in soup.findAll('tr')]
        res = list(filter(None, souplist))
        teamlist = []
        for h in res:
            if ("team" in str(h)) == True:
                teamlist.append(h)
        teamlist = teamlist[0:3]
        team = []
        for h in teamlist:
            a = word_finder_team(str(h))
            a = a.replace('">','')
            a = a.replace('</a>','')
            team.append(a)
        teams.append(team)
    teams.insert(0,'N/A')
    teams.pop()
    coldict = {'year': years, 'top_3_teams_prev_year': teams} 
    top_3_teams = pd.DataFrame(coldict)
    return top_3_teams

In [8]:
def count_past_wins(start_year, end_year, classlist, df, race):
    '''
    Counts number of wins, given a race, per rider 
    '''
    if race == 'giro':
        span = [x for x in range(1909,2022)]
    elif race == 'tour':
        span = [x for x in range(1903,2022)]
    elif race == 'vuelta':
        span = [x for x in range(1935,2022)]
    prev = []
    for n in range(len(df)):
        year = df.loc[n,'year']
        rider = df.loc[n,'rider']
        year_index = span.index(year)
        name = classlist[:year_index]
        #print(name)
        past = name.count(rider)
        prev.append(past)
    return prev

In [9]:
def clean_time(text):
    '''
    Takes a time in form hh:mm:ss or mm:ss
    returns seconds as an integer
    '''
    if ':' in text:
        times = text.split(':')
        if len(times) == 3:
            try:
                total_time = (int(times[0])*60*60) + (int(times[1])*60) + (int(times[2]))
            except ValueError:
                total_time = 'error '+text
        elif len(times) == 2:
            try:
                total_time = (int(times[0])*60) + (int(times[1]))
            except ValueError:
                total_time = 'error '+text
        else:
            total_time = 'error '+text
    else:
        total_time = int(text)
    return total_time

In [10]:
vuelta_years = [x for x in range(1935,2022)]
tour_years = [x for x in range(1903,2022)]
giro_years = [x for x in range(1909,2022)]

# Data Scraping

## Scraping data for each race in time range specified

In [11]:
## Running the race data function to create a dataframe of Tour data 1960-2018
tdf_rank = race_data_multiple_years(1960,2018,'tour-de-france')

In [12]:
## Running the race data function to create a dataframe of Giro data 1960-2018
giro_rank = race_data_multiple_years(1960,2018,'giro-d-italia')

tdf_rank = pd.merge(tdf_rank, giro_rank.add_suffix('_giro'),  how='left', left_on=['year','rider'], right_on=['year_giro','rider_giro']).drop(columns= ['year_giro','rider_giro','age_giro'])


In [13]:
giro_winners = list(giro_rank[giro_rank['rank']=='1'].iloc[:,2])

In [14]:
giro_years = list(giro_rank[giro_rank['rank']=='1'].iloc[:,0])

In [15]:
tdf_rank['rank_giro'] = tdf_rank['rank_giro'].fillna(200)
tdf_rank['team_giro'] = tdf_rank['team_giro'].fillna('Unknown')

In [16]:
## Checking the data is in the correct format
tdf_rank.head()

,year,rank,rider,age,team,time,rank_giro,team_giro,time_giro
0,1960,1,NENCINI Gastone,30,Italy,0,2,Italy,0:28
1,1960,2,BATTISTINI Graziano,24,Italy,5:02,20,Italy,41:13
2,1960,3,ADRIAENSENS Jan,28,Belgium,10:24,12,Belgium,19:32
3,1960,4,JUNKERMANN Hans,26,West-Germany,11:21,14,West-Germany,30:01
4,1960,5,PLANCKAERT Joseph,26,Belgium,13:02,200,Unknown,NaN


In [17]:
## Running the race data function to create a dataframe of Vuelta data 1960-2018
vuelta_rank = race_data_multiple_years(1960,2018,'vuelta-a-espana')

1978


In [18]:
## Filling in the data for 1978, the year errored for the Vuelta

link = 'https://www.procyclingstats.com/race/vuelta-a-espana/1978/gc/'
result = requests.get(link)
soup = BeautifulSoup(result.text, 'html.parser')

## Ranks
rank = [x.td for x in soup.findAll('tr')]
rank_nonone = list(filter(None, rank))
ranks = []
for n in rank_nonone:
    a = clean_number(str(n))
    ranks.append(a)
index_from = [i for i, n in enumerate(ranks) if n == '1'][0]
if [i for i, n in enumerate(ranks) if n == '1'][1] - index_from < 20:
    index_to = [i for i, n in enumerate(ranks) if n == '1'][1]
else:
    index_to = [i for i, n in enumerate(ranks) if n == '1'][1]
ranks = ranks[index_from : index_to]

## Names
names = [x.a for x in soup.findAll('tr')]
names_nonone = list(filter(None, names))
rider = []
for n in names_nonone:
    if ("rider" in str(n)) == True:
        a = word_finder_team(str(n))
        a = a.replace('">','')
        a = a.replace('</a>','')
        rider.append(a)
rider = rider[index_from : index_to]

## Times
time = [x.text for x in soup.findAll('td', class_='time ar')]
time = time[index_from : index_to] 
new_times = []
for t in time: 
    a = t.replace(',,','error')
    a = t.replace(',','')
    a = a.replace(' ','')
    if len(a) > 8:
        middle = int(len(a)/2)
        a = a[:middle]
        new_times.append(a)
    else:
        new_times.append(a)

finish_time = new_times.pop(0)
new_times.insert(0,'0')
time = []
for t in new_times: 
    if len(t) > 7:
        middle = int(len(t)/2)
        t = t[:middle]
        time.append(t)
    else:
        time.append(t)

## Teams
team = [x.text for x in soup.findAll('td', class_='cu600')]
team = team[index_from : index_to]

## Ages
age = [x.text for x in soup.findAll('td', class_="age hide")]
age = age[index_from : index_to]

## Year column
years = [1978 for x in range(len(ranks))]

coldict = {'year': years, 'rank': ranks, 'rider': rider, 'age':age, 'team':team, 'time':time} 
vuelta1978 = pd.DataFrame(coldict)

print(len(years),len(ranks),len(rider),len(age),len(team),len(time))

64 64 64 64 64 64


In [19]:
## Joining this data to the Vuelta dataframe if it is meant to include year 1978
if vuelta_rank['year'].min() <= 1978:
    vuelta_rank = pd.concat([vuelta_rank, vuelta1978])

In [20]:
## Mering Vuelta and Tour data
tdf_rank = pd.merge(tdf_rank, vuelta_rank.add_suffix('_vuelta'),  how='left', left_on=['year','rider'], right_on=['year_vuelta','rider_vuelta']).drop(columns= ['year_vuelta','rider_vuelta','age_vuelta'])

In [21]:
tdf_rank['rank_vuelta'] = tdf_rank['rank_vuelta'].fillna(200)
tdf_rank['team_vuelta'] = tdf_rank['team_vuelta'].fillna('Unknown')

In [22]:
vuelta_winners = list(vuelta_rank[vuelta_rank['rank']=='1'].iloc[:,2])

In [23]:
vuelta_years = list(vuelta_rank[vuelta_rank['rank']=='1'].iloc[:,0])

## Scraping top 3 teams of the previous year

In [24]:
## Using top 3 teams function to scrape top 3 teams per year to a dataframe
top_3_teams = top3teamsprev(1960,2018,'tour-de-france')

In [25]:
## Merging this data with scraped Tour data on year
tdf_ranks = tdf_rank.merge(top_3_teams, how='left', on=['year'])

# Importing dataset of stage wins to enrich scraped data

In [26]:
## Importing found dataset
stages = pd.read_csv('stages_TDF.csv')

In [27]:
## Formatting the name to match other data
stages['Winner'] = stages['Winner'].apply(standardise_name)

In [28]:
## Creating a year column from the date and making it an integer
stages['year'] = [x[:4] for x in stages['Date']]
stages['year'] = stages['year'].astype(int)

In [29]:
## Rename columns ready to merge with main dataframe
stages.rename(columns={'Winner':'rider','Distance':'distance','Type':'type'}, inplace=True)

In [30]:
## Creating a dataframe of average win distance per rider by grouping by rider and averaging the win times
average_win_distance = stages.groupby(['rider'],as_index = False)['distance'].mean()

In [31]:
## Merging average win distance with main dataframe on rider name to add this as a column
tdf_ranks = tdf_ranks.merge(average_win_distance, how='left', on = 'rider')
## For riders who have not won a stage, filling 0
tdf_ranks['distance'] = tdf_ranks['distance'].fillna(0)

In [32]:
## Creating dataframes of mountain stages and time trial stage types
mountain_stages = stages[stages['type'].str.contains('ountain')]
time_trial = stages.loc[(stages['type'].isin(['Individual time trial','Mountain time trial']))]

In [33]:
## Checking overall results for a well-known cyclist for accuracy
tdf_ranks[tdf_ranks['rider']=='FROOME Chris']

,year,rank,rider,age,team,time,rank_giro,team_giro,time_giro,rank_vuelta,team_vuelta,time_vuelta,top_3_teams_prev_year,distance
5496,2008,82,FROOME Chris,23,Barloworld,2:22:33,200,Unknown,NaN,200,Unknown,NaN,"[Discovery Channel , Caisse d'Epargne, CSC Pro...",148.071429
6052,2012,2,FROOME Chris,27,Sky Procycling,3:21,200,Unknown,NaN,4,Sky Procycling,10:16,"[Team Garmin - Cervélo, Leopard Trek, AG2R La ...",148.071429
6204,2013,1,FROOME Chris,28,Sky Procycling,0,200,Unknown,NaN,200,Unknown,NaN,"[RadioShack - Nissan, Sky Procycling, BMC Raci...",148.071429
6537,2015,1,FROOME Chris,30,Team Sky,0,200,Unknown,NaN,200,Unknown,NaN,"[AG2R La Mondiale, Belkin-Pro Cycling Team, Mo...",148.071429
6697,2016,1,FROOME Chris,31,Team Sky,0,200,Unknown,NaN,2,Team Sky,1:23,"[Movistar Team, Team Sky, Tinkoff - Saxo]",148.071429
6871,2017,1,FROOME Chris,32,Team Sky,0,200,Unknown,NaN,1,Team Sky,0,"[Movistar Team, Team Sky, BMC Racing Team]",148.071429


# Adding calculated columns to the Dataframe

In [34]:
## Target column: putting ranks into 6 categories
## This will be the MAIN target

target = []
for r in tdf_ranks['rank']:
    r = int(r)
    if r < 4:
        target.append('podium')
    elif r < 25 and r > 3:
        target.append('4-25')
    elif r < 50 and r > 24:
        target.append('25-50')
    elif r < 75 and r > 49:
        target.append('50-75')
    elif r < 100 and r > 74:
        target.append('75-100')
    else:
        target.append('below 100')
        
tdf_ranks['target'] = target

In [35]:
## A second target column with fewer categories

target2 = []
for r in tdf_ranks['rank']:
    r = int(r)
    if r < 4 :
        target2.append('podium')
    elif r < 26 and r > 3:
        target2.append('4-25')
    elif r < 50 and r > 25:
        target2.append('26-50')
    elif r < 75 and r > 49:
        target2.append('51-75')
    else:
        target2.append('below 75')
        
tdf_ranks['target2'] = target2

In [36]:
## A third target with podium and non podium

target3 = []
for r in tdf_ranks['rank']:
    r = int(r)
    if r < 4:
        target3.append('podium')
    else:
        target3.append('not podium')
        
tdf_ranks['podium'] = target3

In [37]:
## Rank last year in the tour calculated from matching rider name in a sub dataframe of previous year results

prev_rank = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        prev_r = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr-1)]['rank'].item()
        prev_rank.append(prev_r)
    except ValueError:
        prev_rank.append(200)
tdf_ranks['last_year_tour_rank'] = prev_rank

In [38]:
## Rank 2 years ago in the tour calculated from matching rider name in a sub dataframe of results 2 years ago

prev_2rank = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        prev_r = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr-2)]['rank'].item()
        prev_2rank.append(prev_r)
    except ValueError:
        prev_2rank.append(200)
tdf_ranks['last_2year_tour_rank'] = prev_2rank

In [39]:
## This year Giro rank calculated from matching rider name and year to the giro rank column

prev_grank = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        prev_r = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr)]['rank_giro'].item()
        prev_grank.append(prev_r)
    except ValueError:
        prev_grank.append(200)
tdf_ranks['this_year_giro_rank'] = prev_grank

In [40]:
## Last year Giro rank calculated from matching rider name and last year to the giro rank column
## value for last year

prev_g2rank = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        prev_r = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr-1)]['rank_giro'].item()
        prev_g2rank.append(prev_r)
    except ValueError:
        prev_g2rank.append(200)
tdf_ranks['last_year_giro_rank'] = prev_g2rank

In [41]:
## Last year Vuelta rank calculated from matching rider name and last year to the Vuelta rank column

prev_vrank = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        prev_r = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr-1)]['rank_vuelta'].item()
        prev_vrank.append(prev_r)
    except ValueError:
        prev_vrank.append(200)
tdf_ranks['last_year_vuelta_rank'] = prev_vrank

In [42]:
## Stage wins last year calculated by finding a subsection of the stages dataframe 
## containing all stages won by a rider that year, and finding its lengh

prev_wins = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        n_wins = len(stages.loc[(stages['rider'] == name) & (stages['year'] == yr-1)])
        prev_wins.append(n_wins)
    except ValueError:
        prev_wins.append(0)
tdf_ranks['wins_last_year'] = prev_wins

In [43]:
## Number of Giro wins per rider calculated by finding a subsection of the ranks dataframe 
## containing all Giros won by a rider that year, and finding its lengh, else appending 0

prev_wins_giro = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    gir_in = giro_years.index(yr)
    giro_winners_cut = giro_winners[:gir_in]
    if name in giro_winners_cut:
        prev_wins_giro.append(giro_winners.count(name))   
    else:
        prev_wins_giro.append(0)
tdf_ranks['prev_wins_giro'] = prev_wins_giro

In [44]:
## Number of Vuelta wins per rider calculated by finding a subsection of the ranks dataframe 
## containing all Vueltas won by a rider that year, and finding its lengh, else appending 0

prev_wins_vuelta = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    vue_in = vuelta_years.index(yr)
    vuelta_winners_cut = vuelta_winners[:vue_in]
    if name in vuelta_winners_cut:
        prev_wins_vuelta.append(vuelta_winners.count(name))   
    else:
        prev_wins_vuelta.append(0)
tdf_ranks['prev_wins_vuelta'] = prev_wins_vuelta

In [45]:
## How many times a rider has won best young rider

## Importing scraped dataset (in another notebook; code slowing down this notebook)
tourwins = pd.read_csv('tourdf.csv')
## Creating a subset of the dataframe to show best young rider
young = tourwins[['year','young']]
## Adding best young rider column to main dataframe
tdf_ranks = tdf_ranks.merge(young, how='left', on = 'year')

prev_young = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        n_wins = len(young.loc[(young['young'] == name) & (young['year'] < yr)])
        prev_young.append(n_wins)
    except ValueError:
        prev_young.append(0)
tdf_ranks['prev_young'] = prev_young

In [46]:
## Number of mountain stages won in the past 3 years calculated by finding a subsection of the mountain stages dataframe 
## containing all mountain stages won by a rider in the past 3 years, and finding its lengh, else appending 0

prev_mountains = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        n_wins = len(mountain_stages.loc[(mountain_stages['rider'] == name) & (mountain_stages['year'] < yr) & (mountain_stages['year'] > yr-4)])
        prev_mountains.append(n_wins)
    except ValueError:
        prev_mountains.append(0)
tdf_ranks['prev_mountains_3y'] = prev_mountains

In [47]:
## Number of time trial stages won in the past 3 years calculated by finding a subsection of the time trial stages dataframe 
## containing all time trial stages won by a rider in the past 3 years, and finding its lengh, else appending 0

prev_timetrial = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        n_wins = len(time_trial.loc[(time_trial['rider'] == name) & (time_trial['year'] < yr) & (time_trial['year'] > yr-4)])
        prev_timetrial.append(n_wins)
    except ValueError:
        prev_timetrial.append(0)
tdf_ranks['prev_timetrial_3y'] = prev_timetrial

## Creating target columns

In [48]:
## Target categories for each rider last year in the tour calculated using last year's rank

targetly = []
for r in tdf_ranks['last_year_tour_rank']:
    r = int(r)
    if r < 4:
        targetly.append('podium')
    elif r < 25 and r > 3:
        targetly.append('4-25')
    elif r < 50 and r > 24:
        targetly.append('25-50')
    elif r < 75 and r > 49:
        targetly.append('50-75')
    elif r < 100 and r > 74:
        targetly.append('75-100')
    else:
        targetly.append('below 100')
        
tdf_ranks['target_lastyr'] = targetly

In [49]:
## Target 2 categories for each rider last year in the tour calculated using last year's rank

target2ly = []
for r in tdf_ranks['last_year_tour_rank']:
    r = int(r)
    if r < 4 :
        target2ly.append('podium')
    elif r < 26 and r > 3:
        target2ly.append('4-25')
    elif r < 50 and r > 25:
        target2ly.append('26-50')
    elif r < 75 and r > 49:
        target2ly.append('51-75')
    else:
        target2ly.append('below 75')
        
tdf_ranks['target_2lastyr'] = target2ly

In [50]:
## Podium category for each rider last year in the tour calculated using last year's rank

targetply = []
for r in tdf_ranks['last_year_tour_rank']:
    r = int(r)
    if r < 4:
        targetply.append('podium')
    else:
        targetply.append('not podium')
        
tdf_ranks['podium_lastyr'] = targetply

## Finishing time columns

In [51]:
## Cleaning the 'time' column to seconds by applying clean_time function

tdf_ranks['time_seconds'] = tdf_ranks['time'].apply(clean_time)

In [52]:
## Highlighting the errors in the new time column

tdf_ranks[tdf_ranks['time_seconds'].str.contains('error')==True]

,year,rank,rider,age,team,time,rank_giro,team_giro,time_giro,rank_vuelta,...,prev_wins_giro,prev_wins_vuelta,young,prev_young,prev_mountains_3y,prev_timetrial_3y,target_lastyr,target_2lastyr,podium_lastyr,time_seconds
3796,1996,43,GUALDI Mirko,28,Polti,743:,200,Unknown,NaN,200,...,0,0,ULLRICH Jan,0,0,0,below 100,below 75,not podium,error 743:


In [53]:
## Index of the error
rindex = tdf_ranks.index[tdf_ranks['time_seconds'].str.contains('error')==True].item()

In [54]:
## Imputing the mean of the times above and below
avg_time = round(((tdf_ranks.loc[rindex-1,'time_seconds'] + tdf_ranks.loc[rindex+1,'time_seconds'])/2))
tdf_ranks.loc[rindex,'time_seconds'] = avg_time

In [55]:
## Calculating time last year using the new cleaned time column 

prev_time = []
for n in range(len(tdf_ranks)):
    name = tdf_ranks.loc[n,'rider']
    yr = tdf_ranks.loc[n,'year']
    try:
        timely = tdf_ranks.loc[(tdf_ranks['rider'] == name) & (tdf_ranks['year'] == yr-1)]['time_seconds'].item()
        prev_time.append(timely)
    except ValueError:
        prev_time.append(40000)
tdf_ranks['time_last_year'] = prev_time

## Team Membership

In [56]:
## Making sure there are no duplicate rows for analysis

tdf_ranks.drop_duplicates(subset=['year', 'rider', 'team'], inplace=True)

In [57]:
## For each team, calculating whether they share half or more riders with a team the next year
## If so, the second team will be changed to the name of the first
## This ensures accuracy of 'in top 3 teams last year', as many teams change name and sponsors year on year 

for year in range(1961,2018):
    from collections import Counter
    
    ## Finding the index of the first instance of each year
    yrindex = tdf_ranks[tdf_ranks.year==year].first_valid_index()
    lastyrindex = tdf_ranks[tdf_ranks.year==year-1].first_valid_index()
    ## Finding top 3 teams of each year
    yr_teams = tdf_ranks.iloc[lastyrindex]['top_3_teams_prev_year']
    prev_yr_teams = tdf_ranks.iloc[yrindex]['top_3_teams_prev_year']
    ## If both years have a top 3, iterates through riders and appends then to a list for their team
    if len(prev_yr_teams) == 3 and len(yr_teams) == 3:
        team_riders = []
        prev_year = tdf_ranks[tdf_ranks['year']==year-1]
        this_year = tdf_ranks[tdf_ranks['year']==year]
        t1 = [prev_yr_teams[0]]
        t2 =[prev_yr_teams[1]]
        t3=[prev_yr_teams[2]]
        for r in prev_year['rider']:
            try:
                r_team = prev_year.loc[prev_year['rider'] == r]['team'].item()
                if r_team in prev_yr_teams[0]:
                    t1.append(r)
                elif r_team in prev_yr_teams[1]:
                    t2.append(r)
                elif r_team in prev_yr_teams[2]:
                    t3.append(r)
                else:
                    continue
            except ValueError:
                continue
        ## Iterates through riders in the top 3 teams last year, 
        ## and if they are in any of those top 3 teams this year a counter for each team is increased
        ## else they are added to a new list with their new team
        nt1 = 0
        nt2 = 0
        nt3 = 0
        newt1 = []
        newt2 = []
        newt3 = []
        for r in this_year['rider']:
            if r in t1:
                if this_year.loc[this_year['rider'] == r]['team'].item() == t1[0]:
                    nt1 += 1
                else:
                    newt1.append(this_year.loc[this_year['rider'] == r]['team'].item())
            elif r in t2:
                if this_year.loc[this_year['rider'] == r]['team'].item() == t2[0]:
                    nt2 += 1
                else:
                    newt2.append(this_year.loc[this_year['rider'] == r]['team'].item())
            elif r in t3:
                if this_year.loc[this_year['rider'] == r]['team'].item() == t3[0]:
                    nt3 += 1
                else:
                    newt3.append(this_year.loc[this_year['rider'] == r]['team'].item())
            else:
                continue
        ## If 3 or more riders have gone to a new team, looks through they new team names
        ## If 3 or more are identical, this is considered the same team as last year
        ## This team is then changed to the old team name
        for a in [[newt1,t1,nt1], [newt2,t2,nt2], [newt3,t3,nt3]]:
                if len(a[0]) >= a[2] and len(a[0]) >= 3:
                    data = Counter(a[0])
                    topteam = data.most_common(1)[0][0]
                    result = a[0].count(topteam)
                    if result >= 3:
                        i_list = tdf_ranks.index[(tdf_ranks['team'] == topteam) & (tdf_ranks['year'] == yr)].tolist()
                        for i in i_list:
                            tdf_ranks.at[i,'team'] = a[1][0]
                            ttt = tdf_ranks.at[i,'top_3_teams_prev_year']
                            tdf_ranks.at[i,'top_3_teams_prev_year'] = [a[1][0] if x == topteam else x for x in ttt]
                else:
                    continue

In [58]:
tdf_ranks.at[6941,'top_3_teams_prev_year']

['Movistar Team', 'Team Sky', 'BMC Racing Team']

In [59]:
pd.set_option('display.max_columns', None)
tdf_ranks.iloc[4580:4650]

,year,rank,rider,age,team,time,rank_giro,team_giro,time_giro,rank_vuelta,team_vuelta,time_vuelta,top_3_teams_prev_year,distance,target,target2,podium,last_year_tour_rank,last_2year_tour_rank,this_year_giro_rank,last_year_giro_rank,last_year_vuelta_rank,wins_last_year,prev_wins_giro,prev_wins_vuelta,young,prev_young,prev_mountains_3y,prev_timetrial_3y,target_lastyr,target_2lastyr,podium_lastyr,time_seconds,time_last_year
4595,2002,65,KONEČNÝ Tomáš,28,Domo - Farm Frites,1:53:26,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",0.000,50-75,51-75,not podium,200,200,200,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,6806,40000
4596,2002,66,MORENI Cristian,29,Alessio,1:54:17,28,Alessio,30:10,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",0.000,50-75,51-75,not podium,200,200,28,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,6857,40000
4597,2002,67,BESSY Frédéric,30,Crédit Agricole,1:58:58,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",0.000,50-75,51-75,not podium,119,200,200,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,7138,10862
4598,2002,68,MILLAR David,25,Cofidis,1:59:51,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",115.625,50-75,51-75,not podium,200,62,200,200,200,0,0,0,BASSO Ivan,0,0,1,below 100,below 75,not podium,7191,40000
4599,2002,69,PADRNOS Pavel,31,US Postal Service,2:03:10,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",0.000,50-75,51-75,not podium,200,85,200,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,7390,40000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660,2002,130,MCEWEN Robbie,30,Lotto - Adecco,3:03:30,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",186.000,below 100,below 75,not podium,200,114,200,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,11010,40000
4661,2002,131,SVORADA Ján,33,Lampre - Daikin,3:03:30,200,Unknown,NaN,108,Lampre - Daikin,3:03:06,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",208.500,below 100,below 75,not podium,129,200,200,200,200,1,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,11010,11858
4662,2002,132,BALDATO Fabio,34,Fassa Bortolo,3:04:07,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",190.500,below 100,below 75,not podium,81,200,200,79,200,0,0,0,BASSO Ivan,0,0,0,75-100,below 75,not podium,11047,7910
4663,2002,133,DE GROOT Bram,27,Rabobank ProTeam,3:04:44,200,Unknown,NaN,200,Unknown,NaN,"[Kelme - Costa Blanca, O.N.C.E. - Eroski, Team...",0.000,below 100,below 75,not podium,200,200,200,200,200,0,0,0,BASSO Ivan,0,0,0,below 100,below 75,not podium,11084,40000


In [60]:
print(tdf_ranks.at[6941,'top_3_teams_prev_year'][0])
print(tdf_ranks.at[6941,'top_3_teams_prev_year'][1])

Movistar Team
Team Sky


In [61]:
## Now the teams have been amended, we can calculate whether a rider was in a top 3 team last year
## Iterates through riders and see if their team is in list of top 3
winningteamly = []
for n in range(len(tdf_ranks)):
    try:
        if tdf_ranks.iloc[n,4] in tdf_ranks.iloc[n,12]:
            winningteamly.append(1)
        else:
            winningteamly.append(0)
    except KeyError:
        print(n)
tdf_ranks['in_top_3_teams_prev_year'] = winningteamly

In [62]:
tdf_ranks['rank'] = tdf_ranks['rank'].astype(int)
tdf_ranks['rank_giro'] = tdf_ranks['rank_giro'].astype(int)
tdf_ranks['rank_vuelta'] = tdf_ranks['rank_vuelta'].astype(int)
tdf_ranks['age'] = tdf_ranks['age'].astype(int)
tdf_ranks['distance'] = tdf_ranks['distance'].astype(int)
tdf_ranks['time_seconds'] = tdf_ranks['time_seconds'].astype(int)
tdf_ranks['last_year_tour_rank'] = tdf_ranks['last_year_tour_rank'].astype(int)
tdf_ranks['last_year_giro_rank'] = tdf_ranks['last_year_giro_rank'].astype(int)
tdf_ranks['this_year_giro_rank'] = tdf_ranks['this_year_giro_rank'].astype(int)
tdf_ranks['last_year_vuelta_rank'] = tdf_ranks['last_year_vuelta_rank'].astype(int)
tdf_ranks['last_2year_tour_rank'] = tdf_ranks['last_2year_tour_rank'].astype(int)

## Exporting data to csv

In [63]:
## ready to import for cleaning
tdf_ranks.to_csv('tdf_capstone_data.csv')